# IMPORT

In [1]:
from __future__ import annotations
from bqskit import Circuit, compile
from bqskit.compiler.machine import MachineModel
from bqskit.ir.gates import RXXGate, RXGate, RZGate,RYGate,RZZGate,HGate,CNOTGate
from transpiler.BQSkit_custom_gate_definitions import VirtualZGate, GPIGate, GPI2Gate,PartialMSGate,FullMSGate
from transpiler.QASM2_reader import load_qasm
from bqskit.ext import bqskit_to_qiskit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit import transpile
from qiskit.transpiler import Target, InstructionProperties
import pennylane as qml
import numpy as np

In [2]:
# Step 1: Define the MS-Gate Machine Model
IONQ_gate_set={PartialMSGate(),GPIGate(),GPI2Gate()}
def transpile_to_ms(circuit: Circuit) -> Circuit:
    """Transpile the given circuit to use only MS gates."""
    model = MachineModel(circuit.num_qudits, gate_set=IONQ_gate_set)
    compiled_circuit = compile(circuit, model, optimization_level=2)
    return compiled_circuit

# Step 2: Create a simple quantum circuit
qc = Circuit(2)
qc.append_gate(HGate(), [0])
qc.append_gate(HGate(), [1])
qc.append_gate(CNOTGate(), [1,0])
qc.append_gate(RXGate(), [1],params=[1.1561])
qc.append_gate(HGate(), [0])
qc.append_gate(RZZGate(), [0, 1],params=[1])
qc.append_gate(RYGate(), [1],params=[3])
qc.append_gate(RXXGate(), [0,1],params=[2.14584545])

    
qc_qiskit = bqskit_to_qiskit(qc)

# Step 3: Transpile the circuit to MS gates
ms_circuit = transpile_to_ms(qc)

In [3]:
#SAVE QASM
ms_circuit.save("circuit.qasm")
print(qc_qiskit.draw(output='text'))

# Initialize the latest AerSimulator
simulator = AerSimulator(method="statevector")

# Transpile the circuit for the simulator
qc_transpiled = transpile(qc_qiskit, simulator)

# Add instruction to save statevector
qc_transpiled.save_statevector()

# Run the circuit in statevector mode
result = simulator.run(qc_transpiled).result()

# Get the statevector
statevector = result.get_statevector()

# Print the statevector
np.abs(np.array(statevector))**2

     ┌───┐┌───┐    ┌───┐                      ┌──────────────┐
q_0: ┤ H ├┤ X ├────┤ H ├──────■───────────────┤0             ├
     ├───┤└─┬─┘┌───┴───┴────┐ │ZZ(1) ┌───────┐│  Rxx(2.1458) │
q_1: ┤ H ├──■──┤ Rx(1.1561) ├─■──────┤ Ry(3) ├┤1             ├
     └───┘     └────────────┘        └───────┘└──────────────┘


array([0.10533642, 0.41539816, 0.12272557, 0.35653985])

In [4]:
# Step 4: Transform to qiskit and print
qiskit_circuit = load_qasm("circuit.qasm")
print(qiskit_circuit.draw(output='text'))


4.247180323249896
2.676383996455
1.1055876696601032
1.1145330881601763
     ┌──────────────┐┌─────────────┐ ┌──────────────┐»
q_0: ┤ Gpi(0.10272) ├┤ Gpi2(3.347) ├─┤ Gpi2(1.7762) ├»
     ├──────────────┤├─────────────┴┐└──────────────┘»
q_1: ┤ Gpi2(3.3094) ├┤ Gpi2(2.3437) ├────────────────»
     └──────────────┘└──────────────┘                »
«     ┌──────────────────────────────────┐┌─────────────┐»
«q_0: ┤0                                 ├┤ Gpi(4.4979) ├»
«     │  Partialms(2.4756,4.8491,1.1056) │├─────────────┤»
«q_1: ┤1                                 ├┤ Gpi(1.0211) ├»
«     └──────────────────────────────────┘└─────────────┘»
«     ┌──────────────────────────────────┐┌─────────────┐┌──────────────┐»
«q_0: ┤0                                 ├┤ Gpi(2.8029) ├┤ Gpi2(3.1416) ├»
«     │  Partialms(4.8211,5.1766,1.1145) │├─────────────┤├──────────────┤»
«q_1: ┤1                                 ├┤ Gpi(5.4717) ├┤ Gpi2(1.8021) ├»
«     └──────────────────────────────────┘└─────────────┘└─

# NOISE

In [8]:
# Convert Qiskit circuit to PennyLane
qml_circuit = qml.from_qiskit(qiskit_circuit)
np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2#check

array([0.10533645, 0.12272555, 0.41539815, 0.35653985])

In [26]:
def get_noisy_counts(qiskit_qc, noise_level_two_qubit, noise_level_one_qubit, readout_error):
    # Convert the Qiskit quantum circuit into a PennyLane circuit
    qml_circuit = qml.from_qiskit(qiskit_qc)

    # Compute the adjusted noise level for single-qubit depolarizing channels in 2Q noise
    noise_level_two_qubit_single_channel = noise_level_two_qubit * 3 / 4

    # Get the number of qubits in the circuit
    number_of_qubits = qiskit_qc.num_qubits

    # Define a QNode using PennyLane's mixed-state simulator (supports noise)
    @qml.qnode(qml.device("default.mixed", wires=number_of_qubits))
    def noisy_circuit():
        # Convert the PennyLane circuit into a tape (sequence of operations)
        tape = qml.transforms.make_tape(qml_circuit)()

        # Iterate through all operations in the circuit
        for op in tape.operations:
            # Apply each gate as a unitary operation
            qml.QubitUnitary(op.parameters[0], wires=op.wires)

            # Apply **1-qubit depolarizing noise** after every 1-qubit gate
            if len(op.wires) == 1:
                qml.DepolarizingChannel(noise_level_one_qubit, wires=op.wires[0])

            # Apply **2-qubit depolarizing noise** after two-qubit (MS) gates
            if len(op.wires) == 2:
                for qubit in op.wires:
                    qml.DepolarizingChannel(noise_level_two_qubit_single_channel, wires=qubit)  # Adjusted probability

        # Apply **readout error (bit flip) before measurement**
        for qubit in range(number_of_qubits):
            qml.BitFlip(readout_error, wires=qubit)

        # Return the probability distribution over computational basis states
        return qml.probs(wires=range(number_of_qubits))

    # Run the noisy circuit simulation and return the result
    noisy_result = noisy_circuit()
    return noisy_result
print("Noisy     simulation result:", get_noisy_counts(qiskit_circuit, noise_level_two_qubit=0.02, noise_level_one_qubit=0.001, readout_error=0.05))

print("Noiseless simulation result:",np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2)#check

Noisy     simulation result: [0.12613542 0.14111787 0.38952243 0.34322429]
Noiseless simulation result: [0.10533645 0.12272555 0.41539815 0.35653985]


UFuncTypeError: ufunc 'absolute' did not contain a loop with signature matching types <class 'numpy.dtypes.StrDType'> -> None